# 🚀 Vietnamese-English Translation Model Training

**Transformer-based Neural Machine Translation: VI → EN**

This notebook trains a **REVERSED** translation model:
- **Source**: Vietnamese 🇻🇳
- **Target**: English 🇬🇧

Using:
- **PhoMT Dataset** (~3M sentence pairs)
- **Modern Transformer** with RMSNorm, SwiGLU, RoPE
- **AdamW + Cosine Annealing Warm Restarts**

---

## 1. ⚙️ Setup Environment

In [ ]:
# Clone repository
!git clone https://github.com/TranKien2005/EV_Translate_Modle_NLP_Project.git
%cd EV_Translate_Modle_NLP_Project

In [ ]:
# Install dependencies
!pip install -q datasets sentencepiece sacrebleu google-generativeai python-dotenv tqdm tensorboard seaborn pyyaml

In [ ]:
# Verify PyTorch and CUDA
import torch
print(f'PyTorch version: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'CUDA device: {torch.cuda.get_device_name(0)}')

In [ ]:
# Create .env file with API keys
GEMINI_API_KEY = "YOUR_GEMINI_API_KEY_HERE"
HF_TOKEN = "YOUR_HF_TOKEN_HERE"

with open('.env', 'w') as f:
    f.write(f'GEMINI_API_KEY={GEMINI_API_KEY}\n')
    f.write(f'HF_TOKEN={HF_TOKEN}\n')

print('✓ .env file created')

## 2. 🔧 Configure Paths for Kaggle

**IMPORTANT**: Update paths in config file for Kaggle environment.

In [ ]:
# ⚠️ IMPORTANT: Configure paths for Kaggle
# All paths are defined in ONE place: config YAML file

import yaml

CONFIG_FILE = 'config/config_vi_en.yaml'

# Load config
with open(CONFIG_FILE, 'r') as f:
    cfg = yaml.safe_load(f)

# Update paths for Kaggle
cfg['paths'] = {
    'data_dir': '/kaggle/working/data',
    'checkpoint_dir': '/kaggle/working/checkpoints_vi_en',
    'log_dir': '/kaggle/working/logs_vi_en'
}

# Save updated config
with open(CONFIG_FILE, 'w') as f:
    yaml.dump(cfg, f, default_flow_style=False, allow_unicode=True)

print('✓ Config paths updated for Kaggle:')
print(f"  data_dir: {cfg['paths']['data_dir']}")
print(f"  checkpoint_dir: {cfg['paths']['checkpoint_dir']}")
print(f"  log_dir: {cfg['paths']['log_dir']}")

## 3. 📥 Download & Preprocess Data

In [ ]:
# Create data directory
!mkdir -p /kaggle/working/data

# Download PhoMT dataset
!python scripts/download_phomt.py

In [ ]:
# Preprocess data (train tokenizers + create .pt files)
# ⚠️ Uses VI-EN config: Vietnamese as SOURCE, English as TARGET

!python scripts/preprocess_data.py --config config/config_vi_en.yaml

## 4. 🔍 Configuration Check

In [ ]:
import sys
sys.path.insert(0, '.')

from src.config import load_config
from src.models import Transformer

# Load VI-EN config
config = load_config('config/config_vi_en.yaml')

print("="*50)
print("📋 Configuration Summary (VI → EN)")
print("="*50)
print(f"\n🔹 Paths:")
print(f"   data_dir: {config.paths.data_dir}")
print(f"   checkpoint_dir: {config.paths.checkpoint_dir}")
print(f"\n🔹 Model:")
print(f"   d_model: {config.d_model}")
print(f"   layers: {config.num_encoder_layers} enc + {config.num_decoder_layers} dec")
print(f"   d_ff: {config.d_ff}")
print(f"\n🔹 Training:")
print(f"   epochs: {config.epochs}")
print(f"   batch_size: {config.batch_size}")
print(f"   learning_rate: {config.learning_rate}")
print(f"   warmup_steps: {config.warmup_steps}")

In [ ]:
# Count model parameters
model = Transformer(
    src_vocab_size=config.src_vocab_size,
    tgt_vocab_size=config.tgt_vocab_size,
    d_model=config.d_model,
    num_heads=config.num_heads,
    num_encoder_layers=config.num_encoder_layers,
    num_decoder_layers=config.num_decoder_layers,
    d_ff=config.d_ff
)

total_params = sum(p.numel() for p in model.parameters())
print(f"\n📊 Model Parameters: {total_params:,} ({total_params/1e6:.1f}M)")
print(f"📦 Model Size: ~{total_params * 4 / 1024 / 1024:.1f} MB")

## 5. 🏋️ Training

In [ ]:
# Switch to processed data for faster training
import yaml

with open('config/config_vi_en.yaml', 'r') as f:
    cfg = yaml.safe_load(f)

cfg['data']['source'] = 'processed'

with open('config/config_vi_en.yaml', 'w') as f:
    yaml.dump(cfg, f, default_flow_style=False, allow_unicode=True)

print('✓ Config updated to use processed data')

In [ ]:
# Start training!
from src.train import Trainer

trainer = Trainer(config_path='config/config_vi_en.yaml')
trainer.setup()

print("\n" + "="*50)
print("🇻🇳 → 🇬🇧 Vietnamese to English Translation")
print("="*50)

print("\n🚀 Starting training from scratch")
trainer.train()

---

## 📝 Notes

**Model Info (VI → EN):**
- Architecture: Modern Transformer (SwiGLU, RMSNorm, RoPE)
- Direction: Vietnamese → English
- Scheduler: Cosine Annealing with Warm Restarts

**Output Files:**
- Checkpoints: `/kaggle/working/checkpoints_vi_en/`
- Best model: `best_model.pt`
- Tokenizers: `tokenizers/tokenizer_vi.model`, `tokenizers/tokenizer_en.model`